In [1]:
import numpy as np
import pandas as pd
import sqlalchemy as sa
import pickle, json, requests, base64


## Build a scikit-learn model

In [2]:
from sklearn import datasets

iris = datasets.load_iris()
X = iris.data  
Y = iris.target
# print(iris.DESCR)

In [3]:
from sklearn.naive_bayes import GaussianNB
clf = GaussianNB()
clf.fit(X, Y)

GaussianNB(priors=None)

In [4]:
#clf.__dict__

## Generate SQL Code from the Model

In [5]:

def test_ws_sql_gen(pickle_data):
    WS_URL="http://192.168.88.88:1888/model" # "https://sklearn2sql.herokuapp.com/"
    b64_data = base64.b64encode(pickle_data)
    data={"Name":"model1", "PickleData":b64_data , "SQLDialect":"postgresql"}
    r = requests.post(WS_URL, json=data)
    content = r.json()
    # print(content)
    lSQL = content["model"]["SQLGenrationResult"][0]["SQL"]
    return lSQL;


In [6]:
pickle_data = pickle.dumps(clf)
lSQL = test_ws_sql_gen(pickle_data)
print(lSQL[0:2000])

WITH "RawScores" AS 
(SELECT "ADS"."KEY" AS "KEY", CAST(NULL AS FLOAT(53)) AS "LogProba_0", CAST(NULL AS FLOAT(53)) AS "Proba_0", -1.09861228867 + (-0.5 * -0.267793442091 - (0.5 * ("ADS"."Feature_0" - 5.006) * ("ADS"."Feature_0" - 5.006)) / 0.121764003092) + (-0.5 * -0.112109357731 - (0.5 * ("ADS"."Feature_1" - 3.418) * ("ADS"."Feature_1" - 3.418)) / 0.142276003092) + (-0.5 * -1.68535226038 - (0.5 * ("ADS"."Feature_2" - 1.464) * ("ADS"."Feature_2" - 1.464)) / 0.0295040030924) + (-0.5 * -2.64826613862 - (0.5 * ("ADS"."Feature_3" - 0.244) * ("ADS"."Feature_3" - 0.244)) / 0.0112640030924) AS "Score_0", CAST(NULL AS FLOAT(53)) AS "LogProba_1", CAST(NULL AS FLOAT(53)) AS "Proba_1", -1.09861228867 + (-0.5 * 0.495040594659 - (0.5 * ("ADS"."Feature_0" - 5.936) * ("ADS"."Feature_0" - 5.936)) / 0.261104003092) + (-0.5 * -0.500335172182 - (0.5 * ("ADS"."Feature_1" - 2.77) * ("ADS"."Feature_1" - 2.77)) / 0.0965000030924) + (-0.5 * 0.30725034869 - (0.5 * ("ADS"."Feature_2" - 4.26) * ("ADS"."Feature

## Execute the SQL Code

In [7]:
# save the dataset in a database table

#engine = sa.create_engine('sqlite://' , echo=False)
engine = sa.create_engine("postgresql://db:db@localhost/db?port=5432", echo=False)
conn = engine.connect()

lTable = pd.DataFrame(X);
lTable.columns = ['Feature_0', 'Feature_1', 'Feature_2', 'Feature_3']
lTable['KEY'] = range(lTable.shape[0])
lTable.to_sql("INPUT_DATA" , conn,   if_exists='replace', index=False)


In [8]:
sql_output = pd.read_sql(lSQL , conn);


In [9]:
sql_output.sample(12, random_state=1960)

,KEY,Score_0,Score_1,Score_2,Proba_0,Proba_1,Proba_2,LogProba_0,LogProba_1,LogProba_2,Decision
114,114,None,None,None,2.751530e-187,1.042537e-06,9.999990e-01,-4.295713e+02,-13.773854,-1.042537e-06,2
74,74,None,None,None,2.717078e-84,9.984608e-01,1.539184e-03,-1.924176e+02,-0.001540,-6.476503e+00,1
9,9,None,None,None,1.000000e+00,3.203442e-18,1.129895e-25,0.000000e+00,-40.282306,-5.744250e+01,0
88,88,None,None,None,3.078949e-73,9.997963e-01,2.037301e-04,-1.669641e+02,-0.000204,-8.498714e+00,1
25,25,None,None,None,1.000000e+00,3.378233e-16,6.395328e-24,-4.440892e-16,-35.624009,-5.340647e+01,0
5,5,None,None,None,1.000000e+00,1.525989e-14,1.798834e-21,-1.532108e-14,-31.813548,-4.776715e+01,0
48,48,None,None,None,1.000000e+00,2.704142e-18,2.864928e-25,0.000000e+00,-40.451747,-5.651208e+01,0
117,117,None,None,None,3.753023e-285,1.645749e-12,1.000000e+00,-6.549142e+02,-27.132825,-1.645795e-12,2
83,83,None,None,None,2.611248e-134,6.121598e-01,3.878402e-01,-3.075866e+02,-0.490762,-9.471620e-01,1
105,105,None,None,None,4.516547e-271,2.409770e-10,1.000000e+00,-6.224928e+02,-22.146320,-2.409770e-10,2


In [10]:
sql_output.Decision.value_counts()

2    50
1    50
0    50
Name: Decision, dtype: int64

## Scikit-learn Prediction

In [11]:
skl_outputs = pd.DataFrame()
skl_output_key = pd.DataFrame(list(range(X.shape[0])), columns=['KEY']);
skl_output_score = pd.DataFrame(columns=['Score_0', 'Score_1', 'Score_2']);
skl_output_proba = pd.DataFrame(clf.predict_proba(X), columns=['Proba_0', 'Proba_1', 'Proba_2'])
skl_output_log_proba = pd.DataFrame(clf.predict_log_proba(X), columns=['LogProba_0', 'LogProba_1', 'LogProba_2'])
skl_output_decision = pd.DataFrame(clf.predict(X), columns=['Decision'])
skl_output = pd.concat([skl_output_key, skl_output_score, skl_output_proba, skl_output_log_proba, skl_output_decision] , axis=1)
skl_output.sample(12, random_state=1960)


,KEY,Score_0,Score_1,Score_2,Proba_0,Proba_1,Proba_2,LogProba_0,LogProba_1,LogProba_2,Decision
114,114,NaN,NaN,NaN,2.751530e-187,1.042537e-06,9.999990e-01,-4.295713e+02,-13.773854,-1.042537e-06,2
74,74,NaN,NaN,NaN,2.717078e-84,9.984608e-01,1.539184e-03,-1.924176e+02,-0.001540,-6.476503e+00,1
9,9,NaN,NaN,NaN,1.000000e+00,3.203442e-18,1.129895e-25,0.000000e+00,-40.282306,-5.744250e+01,0
88,88,NaN,NaN,NaN,3.078949e-73,9.997963e-01,2.037301e-04,-1.669641e+02,-0.000204,-8.498714e+00,1
25,25,NaN,NaN,NaN,1.000000e+00,3.378233e-16,6.395328e-24,-4.440892e-16,-35.624009,-5.340647e+01,0
5,5,NaN,NaN,NaN,1.000000e+00,1.525989e-14,1.798834e-21,-1.509903e-14,-31.813548,-4.776715e+01,0
48,48,NaN,NaN,NaN,1.000000e+00,2.704142e-18,2.864928e-25,0.000000e+00,-40.451747,-5.651208e+01,0
117,117,NaN,NaN,NaN,3.753023e-285,1.645749e-12,1.000000e+00,-6.549142e+02,-27.132825,-1.644906e-12,2
83,83,NaN,NaN,NaN,2.611248e-134,6.121598e-01,3.878402e-01,-3.075866e+02,-0.490762,-9.471620e-01,1
105,105,NaN,NaN,NaN,4.516547e-271,2.409770e-10,1.000000e+00,-6.224928e+02,-22.146320,-2.409770e-10,2


## Comparing the SQL and Scikit-learn Predictions

In [12]:
sql_skl_join = skl_output.join(sql_output , how='left', on='KEY', lsuffix='_skl', rsuffix='_sql')

In [13]:
sql_skl_join.sample(12)

,KEY_skl,Score_0_skl,Score_1_skl,Score_2_skl,Proba_0_skl,Proba_1_skl,Proba_2_skl,LogProba_0_skl,LogProba_1_skl,LogProba_2_skl,...,Score_0_sql,Score_1_sql,Score_2_sql,Proba_0_sql,Proba_1_sql,Proba_2_sql,LogProba_0_sql,LogProba_1_sql,LogProba_2_sql,Decision_sql
89,89,NaN,NaN,NaN,4.177127e-70,9.999552e-01,4.476646e-05,-1.597513e+02,-4.476747e-05,-1.001405e+01,...,None,None,None,4.177127e-70,9.999552e-01,4.476646e-05,-1.597513e+02,-4.476747e-05,-1.001405e+01,1
76,76,NaN,NaN,NaN,4.950122e-113,9.128444e-01,8.715556e-02,-2.585927e+02,-9.118979e-02,-2.440061e+00,...,None,None,None,4.950122e-113,9.128444e-01,8.715556e-02,-2.585927e+02,-9.118979e-02,-2.440061e+00,1
28,28,NaN,NaN,NaN,1.000000e+00,4.166743e-18,1.830835e-25,0.000000e+00,-4.001940e+01,-5.695986e+01,...,None,None,None,1.000000e+00,4.166743e-18,1.830835e-25,0.000000e+00,-4.001940e+01,-5.695986e+01,0
24,24,NaN,NaN,NaN,1.000000e+00,9.853157e-15,6.061386e-22,-9.769963e-15,-3.225098e+01,-4.885493e+01,...,None,None,None,1.000000e+00,9.853157e-15,6.061386e-22,-9.769963e-15,-3.225098e+01,-4.885493e+01,0
79,79,NaN,NaN,NaN,4.631734e-42,9.999988e-01,1.237942e-06,-9.517564e+01,-1.237943e-06,-1.360206e+01,...,None,None,None,4.631734e-42,9.999988e-01,1.237942e-06,-9.517564e+01,-1.237943e-06,-1.360206e+01,1
102,102,NaN,NaN,NaN,1.049417e-218,1.679154e-07,9.999998e-01,-5.019153e+02,-1.559981e+01,-1.679154e-07,...,None,None,None,1.049417e-218,1.679154e-07,9.999998e-01,-5.019153e+02,-1.559981e+01,-1.679154e-07,2
57,57,NaN,NaN,NaN,1.197348e-34,9.999998e-01,2.332069e-07,-7.810778e+01,-2.332069e-07,-1.527134e+01,...,None,None,None,1.197348e-34,9.999998e-01,2.332069e-07,-7.810778e+01,-2.332069e-07,-1.527134e+01,1
124,124,NaN,NaN,NaN,5.617958e-203,9.717780e-07,9.999990e-01,-4.656988e+02,-1.384414e+01,-9.717784e-07,...,None,None,None,5.617958e-203,9.717780e-07,9.999990e-01,-4.656988e+02,-1.384414e+01,-9.717784e-07,2
51,51,NaN,NaN,NaN,6.188548e-101,9.451696e-01,5.483036e-02,-2.307384e+02,-5.639086e-02,-2.903511e+00,...,None,None,None,6.188548e-101,9.451696e-01,5.483036e-02,-2.307384e+02,-5.639086e-02,-2.903511e+00,1
26,26,NaN,NaN,NaN,1.000000e+00,1.760452e-14,4.114624e-22,-1.754152e-14,-3.167062e+01,-4.924232e+01,...,None,None,None,1.000000e+00,1.760452e-14,4.114624e-22,-1.754152e-14,-3.167062e+01,-4.924232e+01,0


In [14]:
condition = (sql_skl_join.Decision_sql != sql_skl_join.Decision_skl)
sql_skl_join[condition]


,KEY_skl,Score_0_skl,Score_1_skl,Score_2_skl,Proba_0_skl,Proba_1_skl,Proba_2_skl,LogProba_0_skl,LogProba_1_skl,LogProba_2_skl,...,Score_0_sql,Score_1_sql,Score_2_sql,Proba_0_sql,Proba_1_sql,Proba_2_sql,LogProba_0_sql,LogProba_1_sql,LogProba_2_sql,Decision_sql
